
# Guía para reconstrucción de Curvas a partir de su curvatura usando Python

Este documento tiene por objetivo explicar paso por paso el método implementado para la reconstrucción de curvas y sus detalles a raiz de la colaboración con Calsens S.L.

## Base Matemática

Una curva es un objeto unidimensional (determinada por un único parámetro) que tiene unas propiedades geométricas que la describen unívocamente. 
Dada la redundancia matemática hay varias formas equivalentes de describir estos objetos unidimensionales. 

Partiendo de un caso simple y físicamente intuitivo podemos entender cualquier curva (en 3D) como la trayectoria de un objeto moviéndose en un espacio euclídeo. En este caso tenemos una ecuación paramétrica en el que la posición r es una función del tiempo t: $r = g(t)$, donde g es la función del tiempo que describe dicha trayectoria. 

### Método de Frenet Serret: 

Al entender la curva como una trayectoria sabemos que físicamente cualquier movimiento que no sea rectilíneo está sometido a unas fuerzas centrífugas que provocan esta desviación. Cada punto de la trayectoria tiene por tanto además de una posición $\vec{r} = g(t)\vec{u}_{r}$, una velocidad $\vec{v} = g^\prime(t)\vec{u}_{v} = \frac{dr(t)}{dt}\vec{u}_{r} + r\frac{d\vec{u}_{r}}{dt} = $ y una aceleración $\vec{a} = g^{\prime\prime}(t)\vec{u}_{a} = \frac{d^2r(t)}{dt^2}\vec{u}_{v} + v \frac{d\vec{u}_{v}}{dt} =..=\frac{d^2r(t)}{dt^2}\vec{u}_{v} + \frac{v^2}{r}\vec{u}_n  $ donde $\vec{u}_i$ son los vectores unitarios en que marcan la dirección. La aceleración cómo vemos tiene dos componentes: 
 - Una componente debido al **cambio en el módulo de la velocidad** y que tiene la dirección de la misma (tangente a la curva)
 - Una componente debido al **cambio en la dirección de la velocidad** y que tiene una dirección perpendicular a la tangente (llamada dirección normal). 

Considerando un sistema de referencia situado en la partícula en cada momento t y con una velocidad constante ($\frac{d^2r(t)}{dt^2}\vec{u}_{v} = 0$) tenemos que la aceleración sólo tiene la componente normal. 

Si conocemos tanto la velocidad como su cambio en dirección podemos conocer unívocamente la trayectoria de la partícula. En esta premisa se basa el método del triedro Frenet-Serret. Dicho triedro no es más que los vectores que ya conocemos: el tangente $vec{T}$, el normal $\vec{N}$ y el binormal $\vec{B}$ para conocer el plano donde se encuentran los anteriores y no es más que su producto vectorial. Además en lugar de usar el tiempo cómo parámetro se usa la longitud de la curva s. Entonces tenemos el siguiente sistema de ecuaciones para un cambio en el el parámetro longitud de curva s (es decir, según recorremos la curva): 
$$ \left[
 \begin{matrix}
\frac{dT}{ds} \\
\frac{dN}{ds}\\
\frac{dB}{ds}
\end{matrix} \right] 
  = \left[ \begin{matrix}
0 & \kappa(s) & 0 \\
-\kappa(s) & 0 & \tau(s) \\
0 & -\tau(s) & 0 
\end{matrix}\right] \left[ \begin{matrix}
T \\
N \\
B 
\end{matrix}  \right] $$

$$T\cdot N = 0,  \hspace{1em} T \cdot B = 0, \hspace{1em} B \cdot N = 0 \hspace{1em}$$

Donde $\kappa$ es la curvatura: $\kappa(s) = ||\frac{d^2r(s)}{ds^2} || = ||\frac{d\vec{T}(s)}{ds}||$ (al usar s como parámetro la curvatura va en la dirección normal y tanto este como el vector tangente son unitarios)  y $\tau$ la torsión: $\tau = - \vec{N}\cdot \vec{B}^\prime $ que se puede interpretar cómo la medida de la velocidad de rotación del vector binormal: $\vec{B}^\prime = -\tau \vec{N}$.  

Integrando el sistema de ecuaciones matricial anterior obtenemos la trayectoria de la partícula: 
$$r(s) = \int T(s)ds + r_0
$$

Si **asumimos que la curvatura y la torsión son constantes** en el intervalo de integración entonces: 

$$N(i) = \frac{s\cdot \tau(i)B(i-1) -s\cdot \kappa(i)T(i-1)+ N(i-1)}{1 + s^2 \tau^2(i) + s^2 \kappa^2(i)}$$


$$T(i) = s\kappa(i)N(i) + T(i-1)$$


$$ B(i) = -s \tau(i)N(i) + B(i-1) $$


$$r(i) = sT(i) + r(i-1)$$

Donde si hemos divido la curva en N partes iguales de longitud "s", "i" denota la porción correspondiente. Es decir, que conociendo la curvatura, la torsión y al menos un triedro inicial o anterior al que queremos calcular podemos reconstruir la curva completamente. 

### Implementación en Pyhton. 
Para implementar este método se ha creado la siguiente función en Pyhton: 


In [ ]:
def frenet(curvatura, torsion, normal, L):
  if len(curvatura)!=len(torsion):
    return "El vector curvatura y el vector de torsión deben tener el mismo número de elementos"
  T, B , N = [],[],[] # CREACIÓN DE LOS VECTORES VACIOS DEL TRIEDRO Y LA POSICION
  r = []
  s = L/(len(curvatura)) # CALCULO DE LA LONGITUD DE CURVA PARA CADA ITERACION 
  for i in range(0,len(curvatura)): 
    k = curvatura[i] # ASIGNACIÓN DE LA CURVATURA Y TORSIÓN PARA CADA ITERACION
    tau = torsion[i]
    if i == 0:  # CALCULO DEL TRIEDRO Y POSICION PARA LA PRIMERA ITERACION 
      r_0 = np.array([0,0,0]) # PARTIMOS DE 0 
      N_0 = np.append(normal, 0)  # VECTOR EN EL PLANO XY
      N_0 = N_0/np.linalg.norm(N_0) #NORMALIZACIÓN DE  LOS VECTORES
      T_0 = np.array( [ 0,0,1] ) # VECTOR EN DIRECCION Z
      T_0 = T_0/np.linalg.norm(T_0)
      B_0 = np.cross(T_0, N_0) # PRODUCTO VECTORIAL DE T Y N 
      B_0 = B_0/np.linalg.norm(B_0)
    
      r0,T0,B0,N0 = r_0, T_0, B_0, N_0 # ASIGNACION DE LOS VECTORES INICIALES COMO LOS CREADOS ARRIBA
    else: 
      r0,T0,B0,N0 = ri, Ti, Bi, Ni # ASIGNACION DE LOS VECTORES INICIALES COMO LOS DE LA ITERACIÓN ANTERIOR
    
    Ni = (s*tau*B0 -s*k*T0 + N0  )/(1 + s*s*tau*tau + s*s*k*k) # APLICACIÓN DE LAS FÓRMULAS
    Ti = s*k*Ni + T0
    Bi = -s*tau*Ni + B0
    ri = s*Ti + r0

    Ni = Ni/np.linalg.norm(Ni) # NORMALIZACIÓN DE LOS VECTORES CREADOS
    Ti = Ti/np.linalg.norm(Ti)
    Bi = Bi/np.linalg.norm(Bi)

    r.append(ri) # ADICIÓN DE LOS VECTORES CREADOS
    N.append(Ni)
    B.append(Bi)
    T.append(Ti) 
  #CODIGO PARA LA REPRESENTACION, PLOT
  import matplotlib.pyplot as plt
  fig2 = plt.figure(figsize= (12,12))
  ax2 = fig2.add_subplot( projection='3d')
  fig, ax = plt.subplots(3 , 1 , figsize= (12,36))
  for i in range(0,len(r)):
    ax2.scatter(r[i][0], r[i][1], r[i][2])
    ax[0].scatter(r[i][0], r[i][1])
    ax[1].scatter(r[i][1], r[i][2])
    ax[2].scatter(r[i][0], r[i][2])
    
  ax2.plot([r[0][0], r[-1][0]], [r[0][1], r[-1][1]], [r[0][2] , r[-1][2] ])
  ax[0].plot([r[0][0], r[-1][0]], [r[0][1], r[-1][1]] )
  ax[1].plot( [r[0][1], r[-1][1]], [r[0][2] , r[-1][2] ])
  ax[2].plot([r[0][0], r[-1][0]], [r[0][2] , r[-1][2] ])
  ax2.set_title("3D Projection ")
  ax[0].set_title("Proyeccion XY ")
  ax[1].set_title("Proyeccion YZ ")
  ax[2].set_title("Proyeccion XZ ")
  plt.grid(visible = True)  
  return r 

Cómo vemos para reconstruir la curva la función necesita de un vector con los valores de la curvatura en cada punto, los valores de la torsión, un vector normal a la curva y la longitud total de la curva L. (No se pide un vector tangente ya que se asume que el normal está en el plano XY y el tengente en consecuencia está en la dirección del eje Z, por lo que la curva "crecerá" en esta dirección). 

El primer paso es verificar que los vectores curvatura y torsión tienen el mismo tamaño. Seguidamente se definen los vectores posición, tangente, normal y binormal como vectores vacios (r = [  ]). Además se calcula la longitud de cada curva s de cada iteración a partir de la longitud total de la curva L y el número de elementos en el vector de curvatura. 

Seguidamente se entra en el bucle que calcula las posiciones:
 - Para el primer elemento del bucle se toma la posición inicial como 0, el vector normal dado (en el plano XY) y se calcula el binormal junto con el tangente. Una vez se tienen todos los datos simplemente se aplica la fórmula del triedro integrado y se añaden ("append") los valores obtenidos a los vectores posición, tangente, normal y binormal.(Dado que el triedro de vectores es unitario se normaliza antes de añadirlo para curarse en salud).

 - Para el resto de elementos simplemente se coge como vectores iniciales los de la iteración anterior (línea 22) y se repite el proceso de sustitución en las fórmulas y de adición ("append") de los valores a los vectores finales 

Finalmente se pinta la curva en 3D, asi como sus proyecciones en 2D. 
Se devuelve únicamente el vector posición pero si hiciese falta se podrían devolver el resto de vectores (son necesarios para los cálculos por lo que no se pueden omitir su presencia).

El resultado de la función es por tanto un vector (una lista para python) en el que en cada posición de la lista hay un "numpy array" de 3 dimensiones representando las coordenadas x,y,z de la posición de la curva. 

### Método Matricial: 
Este método en lugar de basarse en propiedades de la curva se basa en las propiedades de las transformaciones de coordenadas para un punto. Es un método que tiene en cuenta el que la reconstrucción vaya a producirse de forma iterativa. Es decir, para cada punto de la curva y su siguiente se aplica una traslación junto con dos rotaciones de su sistema original de coordenadas para conseguir la posición del nuevo punto. No obstante esto se hace para car de puntos consecutivos, es decir, del punto 0 al punto 1, pero para ir del punto 0 al punto 2, inevitablemtne hay que pasar por el punto 1. 

Entonces la matriz de cambio del sistema de referencia i+1 al sistema de referencia i es : 

$$A_{i+1} ^i = \left(\begin{matrix} \cos(Θ_i)\cos(\phi_i) & - \sin(Θ_i) & \cos(Θ_i)\sin(\phi_i) & R_i(1-\cos(\phi_i))\cos(Θ_i) \\ 
\sin(Θ_i)\cos(\phi_i) &  \cos(Θ_i) & \sin(Θ_i)\sin(\phi_i) & R_i(1-\cos(\phi_i))\sin(Θ_i)\\
-\sin(\phi_i) & 0 & \cos(\phi_i) & R_i\sin(\phi_i)\\
 0 & 0 & 0 & 1 
\end{matrix} \right) $$

De forma que las coordenas de posición del sistema de referencia i+1 se expresan en el sistema i con la siguiente ecuación: $r^i_{i+1} = A^i_{i+1}r^{i+1}_{i+1}$. 

Nosotros queremos conocer las coordenadas de cada punto en el primer sistema de referencia, por lo que hay que iterar este proceso hasta llegar a $i=0$ empezando por el último punto $i + N = N$, obteniendo un productorio de matrices que representan el paso por los puntos intermedios: 
$$r^0_{N} = A^0_1 \cdot A^1_2....A^{N-1}_{N}r^N_N$$ 

(Es importante notar que los vectores $r^i_j$ son de 4 dimensiones, aunque su última componente es siempre constante e igual a 1 para que el método funcione). Para utilizar este método hacen falta tres datos, el radio de curvatura que une un punto de la curva con el siguiente (se asume constante entre los 2), el ángulo $\theta_i$ que indicaría la dirección en el plano en la que se curva dicha curvatura (es decir el equivalente a la dirección Normal en el método anterior, pero esta dirección se expresa en coordenadas polares con respecto a un sistema de referencia, en nuestro caso $\theta = 0 $ se corresponde con la dirección del eje X), y finalmente otro ángulo $\phi$ que vendría a ser el ángulo que recorre la traslación para un cierto radio de curvatura (en una dirección $\theta$), es decir, se puede relacionar con la distancia total de la traslación (esta traslación no es en línea recta pero a lo largo de una curva con radio de curvatura r).


## Implementación en Pyhton: 
Nuevamente una función calculará la posición de la curva. Los parámetros de entrada son la curvatura, el ángulo en el que se curva dicha curvatura, la longitud total de la curva, y la distancia r que es la distancia de cada fibra al eje neutro (en realidad este parámetro podría omitirse). 


In [ ]:
def Metmatrices(curvatura, angulo, L , r ):
  if len(curvatura)!=len(angulo):
    return "El vector curvatura y el vector de ángulos deben tener el mismo número de elementos"
  eje_n = []
  ang = []
  for j in range(0,len(angulo)): # el ángulo se mide con respecto al punto anterior
    if j ==0: 
      ang.append( angulo[j] )
    else:
      ang.append(angulo[j] - angulo[j-1])  

  p0s, p1s , p2s, p3s, p4s = [],[],[],[],[]
  p0n = np.array([0,0,0,1])
  p1n = np.array([r,0,0,1])
  p2n = np.array([0,r,0,1])
  p3n = np.array([-1*r,0,0,1])
  p4n = np.array([0,-1*r,0,1])
  s = L/len(curvatura)

  for i in range(0,len(curvatura)): 
    radio = 1/(curvatura[i]) 
    theta = ang[i] 
    phi = s * (curvatura[i]) #La curvatura ya está en metros de la primera función
    
    A = np.array([ [np.cos(theta)*np.cos(phi), -np.sin(theta), np.cos(theta)*np.sin(phi), radio*(1-np.cos(phi))*np.cos(theta)],
                   [np.sin(theta)*np.cos(phi), np.cos(theta), np.sin(theta)*np.sin(phi), radio*(1-np.cos(phi))*np.sin(theta)],
                   [-np.sin(phi), 0, np.cos(phi), radio*np.sin(phi)],
                   [0,0,0,1]  ]) # MATRIZ DEL Método
    if i ==(0):
      A0 = A
    else: 
      A0 = A0@A # Cálculo de la matriz A en el sistema de referencia del origen
   
    Op0j = A0@p0n.T   #Cálculo de las posiciones de las fibras y el eje neutro
    Op1j = A0@p1n.T
    Op2j = A0@p2n.T
    Op3j = A0@p3n.T
    Op4j = A0@p4n.T
    
    Op0j = np.delete(Op0j , -1) # Se elimina la dimensión sin información
    Op1j, Op2j, Op3j, Op4j = np.delete(Op1j , -1), np.delete(Op2j , -1) , np.delete(Op3j , -1), np.delete(Op4j , -1)
    p0s.append(Op0j) # Se añaden las posiciones a los vectores finales
    p1s.append(Op1j)
    p2s.append(Op2j)
    p3s.append(Op3j)
    p4s.append(Op4j)
    
    eje_n.append( (Op1j + Op2j + Op3j + Op4j)/4  ) # Se calcula el eje neutro como la ponderación de los 4 sensores. 

  # import matplotlib.pyplot as plt #Si se ponen dentro del loop se pintan progresivamente y queda chulo 
  # fig = plt.figure(figsize= (12,12))
  # ax = fig.add_subplot( projection='3d')
  # for i in range(0,len(p0s)):
  #   ax.scatter(p0s[i][0], p0s[i][1],  p0s[i][2] , c = "black")
  #   ax.scatter(p0s[i][0], p0s[i][1], 0 , c = "r", marker = ".", alpha = 0.3)
  #   #ax.scatter(p0s[i][0], p0s[i][1] ,p0s[i][2] , c = "r", marker = ".")
  #   #ax.scatter(0 ,p0s[i][1] ,p0s[i][2] , c = "r")
  # ax.plot( [ p0s[0][0],p0s[-1][0] ],[ p0s[0][1], p0s[-1][1] ] ,[ p0s[0][2], p0s[-1][2] ])
  # plt.grid(visible = True)
  # ax.set_title("Método de las Matrices")
  # plt.show()
  return np.array(eje_n), p0s, p1s,p2s,p3s,p4s

De igual forma que en el método de Frenet-Serret imponemos la condición de que los datos de curvatura y ángulo deben ser de igual tamaño. 
Seguidamente definimos el vector donde se guardarán los valores da la posición del eje neutro. 

A continuación dado que este método es iterativo, y siempre usa el punto anterior para continuar, las médidas de los ángulos tienen que ser modificadas. Es decir, cada medida del ángulo en este método será la diferencia entre el ángulo para una determinada iteración y la iteración anterior. Así por ejemplo si el ángulo es constante, entonces el primer elemento será no nulo y el resto serán 0, ya que de forma contraria se acumularían los ángulos y las posiciones estarían girando sobre sí mismas contínuamente. Esto es exactamente lo que realiza el loop situado entre las líneas 5 y 9. 

Antes de entrar en el método iterativo, definimos las posiciones de las fibras con respecto al eje neutro siendo este p0s, la primera fibra p1s, la segunda p2s...etc, ya que el método reconstruye también la posición de estas fibras directamente.

Una vez tenemos el primer ángulo y la curvatura procedemos al método iterativo, en el cual todavía nos falta calcular el segundo ángulo y el radio que es simplemente el inverso de la curvatura para cada medida. Dado que las medidas están en radianes, el ángulo phi que falta se calcula sabiendo que la longitud de arco s es simplemente el producto del radio por dicho ángulo: $L_{arco} = s = r\cdot \phi\hspace{1em} → \hspace{1em}\phi_i = s/r = s \cdot \kappa_i$. 

Una vez tenemos ya todos los datos para cada iteración, los sustituimos en la matriz A del método. Una vez sustituida dado que el método es iterativo  calculamos la matriz A0, que sería la posición del resto de puntos en el sistema de referencia del primer punto o del origen. Para el primer punto A0 = A, y para los siguientes es el producto de la matriz A0 anterior por la nueva A, hasta llegar a la A que corresponde a la última medida. 

Para cada medida una vez se obtiene A0 se multiplica por la posición de cada fibra en el sistema de referencia del punto sobre el que se itera (estas posiciones son constantes para todas las fibras). Una vez obtenido el punto, se elimina el último elemento de la posición (recordemos el método usa una matriz de 4 dimensiones, pero hay una dimensión que no da información) y se añade el punto calculado a la lista de posiciones finales. 

APUNTE: el eje neutro se calcula de 2 formas paralelas, primero simplemente como la posición del origen de cada punto en el método iterativo (p0s), y después como el punto ponderado correspondiente a la posición de las otras 4 fibras (llamado eje_n). Parece que p0s es aparentemente un poco más fiable que eje_n. 

El output es por tanto los dos cálculos de las posiciones del eje neutro y la posición de cada fibra. 

# Rotaciones 

Ambos métodos usan secciones transversales de las fibras. Es decir, las fibras están colocadas longitudinalmente a lo largo del objeto, pero en cada punto forman un plano que es transversal a esta dirección longitudinal. El primer plano que forman estas fibras en el inicio del objeto es nuestro plano XY. Por lo tanto el objeto se va construyendo en la dirección positiva del eje Z. No obstante debido a la curvatura y la torsión está dirección se va desviando progresimante por lo que el punto final puede acabar en cualquier lugar. Dadas las geometrías a las que se tiene previsto aplicar este método, se ha creado una función para girar el objeto final de forma que tanto el primer punto como el último estén sobre la misma línea vertical. 

Para ello primero se definen las matrices de rotación puras en los ejes X, Y, Z (con la convención de ángulos de euler https://es.wikipedia.org/wiki/%C3%81ngulos_de_Euler ):

In [ ]:
#Usan los ángulos de Euler (alpha, beta, gamma) = (phi,theta,psi)
def rotx(phi):
  return np.array( [[1,0,0],[0, np.cos(phi), -np.sin(phi)],[0,np.sin(phi),np.cos(phi)]] )
def roty(thet): 
  return np.array([[np.cos(thet),0,np.sin(thet)],[0,1,0],[-np.sin(thet),0,np.cos(thet)]] )
def rotz(gamma): 
  return np.array([[np.cos(gamma),-np.sin(gamma),0],[np.sin(gamma),np.cos(gamma),0],[0,0,1]] )
#MatrizEuler : rotz*rotx*roty 

La función que se ha definido para alinear el primer y último punto es la siguiente: 


In [ ]:
def rotacion(posi):
  pos1, posf = [],[]
  if (posi[-1][0] < 0) and (posi[-1][0]< 0 ): # Cálculo del ángulo  con respecto al plano XZ
    alpha = - np.arcsin(posi[-1][0]/np.sqrt(posi[-1][0]**2 + posi[-1][1]**2) )
  else:
    alpha =  np.arcsin(posi[-1][0]/np.sqrt(posi[-1][0]**2 + posi[-1][1]**2) )
  
  for i in range(0,len(posi)): # Rotacion sobre el eje Z, el punto acaba en el plano XZ
    m = rotz(alpha)@posi[i]
    pos1.append(m)

  if (posi[-1][1] < 0) and (posi[-1][2]< 0 ): # Cálculo del ángulo con respecto al plano YZ (la coordenada Y será nula
    beta =  - np.arcsin(posi[-1][1]/np.sqrt(posi[-1][1]**2 + posi[-1][2]**2) ) # tras la primera rotación 
  else:
    beta = np.arcsin(posi[-1][1]/np.sqrt(posi[-1][1]**2 + posi[-1][2]**2) )
  
  for i in range(0,len(pos1)): # Rotación alrededor del eje Y para colocar el último punto sobre la vertical.
    m2 = rotx(beta)@pos1[i]
    posf.append(m2)
  
  return posf 

Esta función sólo usa el último punto. Calcula primero la proyección de este punto en el eje x, para en conjunción con su coordenada y, calcular el ángulo que forma el último punto del objeto reconstruido con respecto al eje del plano XZ y rotarlo alrededor del eje Z para dejarlo en este plano. 

Hay una pequeña condición sobre el ángulo dependiendo del cuadrante en el que esté el último punto. Es decir, si en el plano XY, el punto se encuentra en el cuadrante correspondiente a X < 0, Y < 0, entonces hay que cambiar el ángulo de signo, para el resto de cuadrantes el ángulo es el calculado ya que si X < 0 el propio ángulo sale ya con el signo correspondiente. 

Una vez calculado el punto, que ahora debe estar en el plano XZ, se calcula nuevamente su posición en este plano con respecto al eje X, para conocer el ángulo que forma con el mismo y rotarlo alrededor del eje Y de forma que el punto acabe en el eje Z, y por tanto vertical al inicial. 


# Interpolación de las deformaciones y Cálculo de la Curvatura: 

Para cada objeto se colocan medidas puntuales de la deformación alrededor del mismo. Sin embargo, para la reconstrucción de los objetos a partir de su curvatura es necesario conocer el valor de la misma alrededor de todo el objeto, para ello se usa una interpolación "cubic spline". Una vez interpoladas las deformaciones se pasa a calcular la curvatura que se obtiene de ellas a partir de la siguiente fórmula: 

 $$\kappa _{app} = - \sum ^N \frac{\epsilon_i }{r_i}\cos\theta_i \bf{j} - \sum ^N \frac{\epsilon_i }{r_i}\sin\theta_i \bf{k} $$

$$\kappa = \frac{2 |\kappa _{app}|}{N}$$
$$ \theta = angle( \bf{\kappa _{app}}, X) $$ 

## Implementación en Python: 



In [ ]:
def CurvaturayA(strains,posiciones, erre, angulos, N,L, fijo):
  import numpy as np 
  e_1 , e_2, e_3, e_4 = [],[],[],[]
  for i in range(0, len(strains[0:])):
    e_1.append(strains[i][0]) , e_2.append(strains[i][1]) , e_3.append(strains[i][2]), e_4.append(strains[i][3])
  
  if fijo==1: 
    e_1 , e_2, e_3, e_4 = np.array(e_1) , np.array(e_2), np.array(e_3), np.array(e_4)
    e_1 , e_2, e_3, e_4 = np.append(0,e_1), np.append(0,e_2), np.append(0,e_3), np.append(0,e_4)
    posiciones = np.append(0,np.array(posiciones))
  elif fijo==2: 
    e_1 , e_2, e_3, e_4 = np.array(e_1) , np.array(e_2), np.array(e_3), np.array(e_4)
    e_1 , e_2, e_3, e_4 = np.append(e_1,0), np.append(e_2,0), np.append(e_3,0), np.append(e_4,0)
    posiciones = np.append(np.array(posiciones), L)
  elif fijo==12: 
    e_1 , e_2, e_3, e_4 = np.array(e_1) , np.array(e_2), np.array(e_3), np.array(e_4)
    e_1 , e_2, e_3, e_4 = np.append(0,e_1), np.append(0,e_2), np.append(0,e_3), np.append(0,e_4)
    e_1 , e_2, e_3, e_4 = np.append(e_1,0), np.append(e_2,0), np.append(e_3,0), np.append(e_4,0)
    posiciones = np.append(0,np.array(posiciones))
    posiciones = np.append(np.array(posiciones), L)

  from scipy import interpolate
  interp_strains1 = interpolate.interp1d(posiciones, e_1, kind='cubic')
  interp_strains2 = interpolate.interp1d(posiciones, e_2, kind='cubic')
  interp_strains3 = interpolate.interp1d(posiciones, e_3, kind='cubic')
  interp_strains4 = interpolate.interp1d(posiciones, e_4, kind='cubic')

  x = np.linspace(posiciones[0],posiciones[-1] ,N)

  e_1p, e_2p, e_3p, e_4p = interp_strains1(x), interp_strains2(x), interp_strains3(x) , interp_strains4(x)
  # if any(np.array(e_1p)==0):
  #   e_1p = np.delete(e_1p, e_1p==0)
  
  import matplotlib.pyplot as plt 
  fig = plt.figure(figsize= (12,12))
  ax = fig.add_subplot( )
  ax.plot(x, e_1p, color = "r" ,label = "e_1", alpha = 0.4)
  ax.scatter(posiciones, e_1)
  ax.scatter(posiciones, e_2)
  ax.scatter(posiciones, e_3)
  ax.scatter(posiciones, e_4)

  ax.plot(x, e_2p, color = "g" ,label = "e_2", alpha = 0.4)
  ax.plot(x, e_3p, color = "b" ,label = "e_3", alpha = 0.4)
  ax.plot(x, e_4p, color = "m" ,label = "e_4", alpha = 0.4)
  ax.set_title("Interpolated Strains ")
  plt.show()

  vector_kapp = []
  vector_k = [] 
  vector_theta = []
  theta_1 , theta_2, theta_3, theta_4 =  angulos[0], angulos[1], angulos[2], angulos[3]
  r1,r2,r3,r4 = erre[0],erre[1],erre[2],erre[3] 
  for i in range(0, len(e_1p)): 
    k_1 , k_2, k_3, k_4 = np.array([ -e_1p[i]/r1*np.cos(theta_1), - e_1p[i]/r1*np.sin(theta_1) ]) , np.array([-e_2p[i]/r2*np.cos(theta_2),  - e_2p[i]/r2*np.sin(theta_2)]), np.array([-e_3p[i]/r3*np.cos(theta_3),  - e_3p[i]/r3*np.sin(theta_3)]), np.array( [-e_4p[i]/r4*np.cos(theta_4),  - e_4p[i]/r4*np.sin(theta_4)])  
    k_app = np.array( k_1 + k_2 + k_3 + k_4 ) 
    k = 2*np.linalg.norm(k_app)/4   

    if k_app[0] == 0 and k_app[1] == 0 : 
      theta = 0 
      continue

    else: 
      ang = (k_app@np.array([1,0])) /  (np.linalg.norm(k_app))
      theta = np.arccos(  (k_app@np.array([1,0])) /  (np.linalg.norm(k_app)) )
      vector_kapp.append(k_app)
      vector_k.append(k)
      vector_theta.append(theta)  
 
  torsion = []
  for i in range(0,len(vector_theta)-1): 
    torsion.append( (vector_theta[i+1]-vector_theta[i])/(len(vector_theta)-1) ) 


  return vector_k[1:], vector_theta[1:], vector_kapp[1:], torsion

El método tiene como inputs: 
 - las deformaciones, cada sensor debe dar sus medidas en forma de columna, por tanto se debe dar una matriz con 4 columnas. 
 - Las posiciones (distancia al origen) de cada sensor. 
 -  la distancia al eje neturo de cada sensor (en forma de lista o vector horizontal), llamado erre,
 -  el ángulo que forman con respecto al eje X (en forma de lista o vector horizontal),
 -  el número N de puntos sobre los que queremos obtener la interpolación (partimos de 8 medidas pero queremos calcular la curvatura a lo largo de 300 puntos por ejemplo),
 -  la longitud total del objeto en metros
 - Un parámetro llamado fijo, que puede ser 1,2, 12, el resto de números no producen ningún efecto. 

 Devuelve como output: 
 - Las curvaturas (escalares, módulo de la aparente) de cada punto interpolado.  
 - Los ángulos de esta curvatura en el plano transversal de cada sección con respecto al eje X también. 
 - La torsión 
 - La curvatura aparente (en lugar de una curvatura escalar y un ángulo es una curvatura vectorial, que por definición tiene una dirección). 


 #### Funcionamiento del código: 

 Primeramente se selecciona cada medida de cada sensor y se separa para tenerla como un único vector. Esto es lo que hace el primer loop.
 
 Seguidamente tenemos los condicionales correspondientes al parámetro fijo: 
 - Fijo=1 significa que el origen del objeto está fijo, es decir, que sus deformaciones en el origen van a ser 0. 
 - Fijo = 2 significa que el final del objeto está fijo, nuevamente sus deformaciones son 0. 
 - Fijo=12 significa que tanto el origen como el final están fijos con deformaciones nulas en los extremos. 

 Cualquier otro número no produce ningún otro resultado.  

 Seguidamente se pasa a interpolar, para ello se usa el método de la librería scipy para vectores de una dimensión: https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html

A continuación se calculan los puntos donde se quieren obtener los valores interpolados, es el vector llamdo x. Estos puntos se equispacian alrededor de la longitud del objeto. 

El siguiente paso del código es graficar las interpolaciones de  las deformaciones de los 4 sensores y sus valores reales. 

Seguidamente se pasa a calcular la curvatura aparente, y después la curvatura escalar y el ángulo a partir de la fórmula mencionada. Es importante saber que los puntos donde el módulo de la curvatura es 0, el ángulo se asigna como 0 de igual forma. 

Una vez se tienen tanto la curvatura aparente como la escalar y el ángulo, se calcula la torsión de forma aproximada simplemente como la diferencia entre el ángulo en un punto y el ángulo en el siguiente punto divido por la distancia entre esos dos puntos. La torsión es en realidad la derivada del ángulo con respecto a la longitud del objeto, pero dado que no tenemos una fórmula explícita para el ángulo no se puede calcular de esta manera. El cálculo es cada vez menos aproximado y más exacto a medida que la distancia entre los puntos se aproxime a 0 (la derivada se define de hecho como este límite).

Por último se devuelven los valores, pero dado que en la torsión se utiliza una resta, se pierde una unidad de tamaño de este vector con respecto al resto, por lo tanto se elimina el primer elemtno y se devuelven el resto de vectores. 
